In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [21]:


# Imports
import numpy as np
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
from datetime import timedelta
import pathway.reducers

output_notebook()

# Load and preprocess dataset
df = pd.read_csv("dataset.csv")
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream_model1.csv", index=False)

# Define schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

# Replay stream
stream = pw.demo.replay_csv("parking_stream_model1.csv", schema=ParkingSchema, input_rate=100)

# Convert timestamp and filter invalid data
fmt = "%Y-%m-%d %H:%M:%S"
stream = stream.with_columns(t=stream.Timestamp.dt.strptime(fmt))
stream = stream.filter(stream.Capacity > 0)

# Pricing parameters
ALPHA = 2.0
BASE_PRICE = 10.0
window_minutes = 5
window_duration = timedelta(minutes=window_minutes)

# Apply temporal smoothing using sliding window
model1_result = stream.with_columns(
    t=stream.t,
    price_raw=BASE_PRICE + ALPHA * (stream.Occupancy / stream.Capacity)
).windowby(
    stream.t,
    window=pw.temporal.sliding(hop=window_duration, duration=window_duration)
).reduce(
    t=pathway.reducers.max(pw.this.t),
    price=pathway.reducers.sum(pw.this.price_raw) / pathway.reducers.count(pw.this.price_raw)
)

# Panel extension
pn.extension()

# Plot function
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=550,
        width=1100,
        title="Model 1 Streaming Price (Smoothed)",
        x_axis_type="datetime",
        y_range=(9, 13)
    )

    fig.line("t", "price", source=source, line_width=2, color="blue", legend_label="Price")

    if "t" in source.data and len(source.data["t"]) > 0:
        indices = list(range(0, len(source.data["t"]), 100))
        subsampled_data = {k: [source.data[k][i] for i in indices] for k in source.data}
        subsampled_source = ColumnDataSource(subsampled_data)
        fig.circle("t", "price", source=subsampled_source, size=6, color="red")

    return fig

# Generate visualization
viz = model1_result.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()




Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [22]:
# Run Pathway pipeline
%%capture --no-display
pw.run()

Output()